In [1]:
import math
import numpy as np
import openrtdynamics2.lang as dy
import openrtdynamics2.py_execute as dyexe
import openrtdynamics2.code_generation_helper as cgh
import openrtdynamics2.targets as tg


In [3]:
#tg.generate_algorithm_code()

In [4]:
import subprocess
import os
import json
from pathlib import Path


In [24]:



class TargetTemplate:
    """
        template for a custom target
    """

    def __init__(self, enable_tracing=False ):

        # those are set via set_compile_results after a system is compiled
        self.compileResults = None
        self.main_command = None
        self._list_of_code_sources = {}
        
        self.files = None

        #
        self._algorithm_code = None

        # list of systems to include
        self._included_systems = []

        self._enable_tracing = enable_tracing

#    def set_compile_results(self, compile_results : CompileResults ):
    def set_compile_results(self, compile_results ):
        
        self.compileResults = compile_results
        self.main_command = compile_results.command_to_execute

#    def include_systems(self, system : SystemLibraryEntry):
    def include_systems(self, system ):
        
        self._included_systems = system

    def add_code_to_include(self, list_of_code_sources = {}):
        
        self._list_of_code_sources = { **self._list_of_code_sources, **list_of_code_sources }

    def get_algorithm_code(self):
        """
            Return only the code that implement the system and all sub systems
        """
        return self._algorithm_code



    def code_gen(self):

        # generate code for the algorithm
        self.manifest, self._algorithm_code = tg.generate_algorithm_code(
            self.compileResults, 
            self._enable_tracing, 
            self._included_systems, 
            self._list_of_code_sources
        )

        self.files = {}
        self.files['simulation_manifest.json'] = json.dumps( self.manifest.export_json(), indent=4 )
        
        # embedd algorithm source code into a template
        sourcecode = self._algorithm_code
        

        # return
        return {
            'files'      : self.files,
            'sourcecode' : sourcecode, # remove this in favor of 'files'
            'manifest' : self.manifest,
            'algorithm_sourcecode' : self._algorithm_code }

    def write_code(self, folder):
            
        for fname, content in self.files.items():
            fullname = os.path.join( folder + '//' + fname )
            
            print('writing file', fullname)
            
            with open( fullname, 'w') as outfile: 
                
                outfile.write( content )
                

    def build(self):
        pass

    def run(self):
        pass



In [50]:
class CppMain(TargetTemplate):
    """
        generate a simple main.cpp programm
    """

    def __init__(self, enable_tracing=False ):
        
        TargetTemplate.__init__(self, enable_tracing)

    def code_gen(self):
        
        res = TargetTemplate.code_gen(self)
        
        headers = '#include <stdio.h>\n'
        
        main_fn = """
int main () {

    simulation system_instance;

    simulation::Inputs inputs;
    simulation::Outputs outputs;
    
    // set const inputs
    inputs.u1 = 1;
    inputs.u2 = 2;
    
    // reset system
    system_instance.step( outputs, inputs, false, false, true );
    
    int i_max = 10;
    
    for (int i = 0; i < i_max; ++i) {
        
        // calculate outputs
        system_instance.step( outputs, inputs, true, false, false );
        
        // update states
        system_instance.step( outputs, inputs, false, true, false );
    
        // print outputs
        printf("%2.3f %2.3f\\n", outputs.y1, outputs.y2);
    } 
    
    return 0;
}
        
        """
        
        main_cpp = headers + '\n' + res['algorithm_sourcecode'] + '\n' + main_fn
        
        self.files['main.cpp'] = main_cpp
        
        
        return res

In [51]:
dy.clear()

system = dy.enter_system()

# define system inputs
u1               = dy.system_input( dy.DataTypeFloat64(1), name='u1', default_value=1.0,  value_range=[0, 25], title="input #1")
u2               = dy.system_input( dy.DataTypeFloat64(1), name='u2', default_value=2.0,  value_range=[0, 25], title="input #2")


y1 = dy.signal()                         # introduce variable y
x = y1 + u1                               # x[k] = y[k] + u[k]
y1 << dy.delay(x, initial_state = 2.0)   # y[k+1] = y[k] + x[k], y[0] = 2.0

# some second output
y2 = y1 + u2

# define output(s)
dy.append_output(y1, 'y1')
dy.append_output(y2, 'y2')

# generate code for Web Assembly (wasm), requires emcc (emscripten) to build
code_gen_results = dy.generate_code(template=CppMain(), folder='tmp/test')

compiling system simulation (level 0)... 
Generated code will be written to tmp/test .
writing file tmp/test//simulation_manifest.json
writing file tmp/test//main.cpp


In [47]:
# list the blocks in the diagram
dy.show_blocks()


-------- list of blocks --------

-----------------------------
Blocks in simulation simulation:
-----------------------------
* '1' (1)
  input signals
    - y1 (double [1]) type proposal: (double [1])
    - u1 (double [1])
  output signals
    - s3 (double [1]) type proposal: (double [1])
* '2' (2)
  input signals
    - s3 (double [1]) type proposal: (double [1])
  output signals
    - y1 (double [1]) type proposal: (double [1])
* '3' (3)
  input signals
    - y1 (double [1]) type proposal: (double [1])
    - u2 (double [1])
  output signals
    - y2 (double [1]) type proposal: (double [1])

  nested subsystems


In [ ]:
# show the manifest

from IPython.display import JSON, Code
JSON(code_gen_results['manifest'], expanded=True)

In [ ]:
# show the generated source code

Code(data=code_gen_results['algorithm_sourcecode'], language='c++')